# Install Kubeflow SDK

In [1]:

!pip install kfp --upgrade --user

    100% |████████████████████████████████| 307kB 2.2MB/s ta 0:00:01
    100% |████████████████████████████████| 133kB 6.3MB/s ta 0:00:01
    100% |████████████████████████████████| 645kB 6.0MB/s ta 0:00:011
    100% |████████████████████████████████| 122kB 8.4MB/s eta 0:00:01
    100% |████████████████████████████████| 112kB 7.3MB/s ta 0:00:01
    100% |████████████████████████████████| 1.6MB 4.1MB/s ta 0:00:01
    100% |████████████████████████████████| 71kB 8.3MB/s ta 0:00:01
    100% |████████████████████████████████| 153kB 7.8MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 10.7MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 8.2MB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 8.2MB/s eta 0:00:01
    100% |████████████████████████████████| 102kB 7.7MB/s a 0:00:01
    100% |████████████████████████████████| 92kB 7.8MB/s ta 0:00:011
    100% |████████████████████████████████| 1.0MB 1.2MB/s ta 0:00:01
    100% |████████████████████████

    100% |████████████████████████████████| 61kB 7.2MB/s ta 0:00:011
    100% |████████████████████████████████| 143kB 6.3MB/s ta 0:00:01
    100% |████████████████████████████████| 102kB 7.8MB/s a 0:00:011
Requirement not upgraded as not directly required: pyasn1-modules>=0.2.1 in /home/blink/anaconda3/lib/python3.7/site-packages (from google-auth<2,>=1.6.1->kfp) (0.2.2)
    100% |████████████████████████████████| 102kB 7.1MB/s a 0:00:011
  Installing build dependencies ... done
Requirement not upgraded as not directly required: attrs>=17.4.0 in /home/blink/anaconda3/lib/python3.7/site-packages (from jsonschema<4,>=3.0.1->kfp) (18.2.0)
Requirement not upgraded as not directly required: wrapt<2,>=1.10 in /home/blink/anaconda3/lib/python3.7/site-packages (from Deprecated<2,>=1.2.7->kfp) (1.10.11)
Requirement not upgraded as not directly required: wheel in /home/blink/anaconda3/lib/python3.7/site-packages (from strip-hints<1,>=0.1.8->kfp) (0.31.1)
Requirement not upgraded as not directly

  The script strip-hints is installed in '/home/blink/.local/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  The scripts dsl-compile, dsl-compile-v2 and kfp are installed in '/home/blink/.local/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You are using pip version 10.0.1, however version 22.1.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


# Creating python functions to wrap components

In [2]:
import kfp
from kfp import dsl

def extract_multiply(adm_no):
    return dsl.ContainerOp(
        name='extract',
        image='python:3.6.8',
        command=['python', '-c'],
        arguments=['with open("/tmp/extract.txt", "a") as file: file.write(str( ({}%100//10)*({}%100%10) )'.format(adm_no, adm_no)],
        file_outputs={
            'data': '/tmp/extract.txt',
        }
    )

def add_static():
    return dsl.ContainerOp(
        name='extract',
        image='python:3.6.8',
        command=['python', '-c'],
        arguments=['with open("/tmp/static.txt", "a") as file: file.write(str({}))'.format(4+5)],
        file_outputs={
            'data': '/tmp/static.txt',
        }
    )

def square(number,pos):
    return dsl.ContainerOp(
        name='power',
        image='python:3.6.8',
        command=['python', '-c'],
        arguments=['with open("/tmp/square.txt", "a") as file: file.write(str( pow(int(str({})[{}]),2) ))'.format( number,pos)],
        file_outputs={
            'data': '/tmp/square.txt',
        }
    )


def add(a, b):
    return dsl.ContainerOp(
        name='add',
        image='python:3.6.8',
        command=['python', '-c'],
        arguments=['with open("/tmp/results.txt", "a") as file: file.write(str({} + {}))'.format(a, b)],
        file_outputs={
            'data': '/tmp/results.txt',
        }
    )


def echo_op(text):
    return dsl.ContainerOp(
        name='echo',
        image='library/bash:4.4.23',
        command=['sh', '-c'],
        arguments=['echo "Result: {}"'.format(text)]
    )

# Task: MLOps
## Create the following 7-Step Kubeflow pipeline in Python using the KFP SDK.



Pipeline Step-1: Take the last two digits of your Student ID and multiple them.
Pipeline Step-2: Add Numbers 4 and 5.
Pipeline Step-3: Add the resulting outputs from Step 1 and Step 2.
Pipeline Step-4: Take the output from Step 3 and suqare the first (tens-)digit
Pipeline Step-5: Take the output from Step-3 and Square the second (ones-)digit
Pipeline Step-6: Add the resulting outputs from Step 4 and Step 5.
Pipeline Step-7: Echo the resultant to logs.

    4*5                4+5
    = 20                = 9
        \               /
         \             /
          \           / 
           \         /
            20+9 =29
            /       \
           /         \
          /           \
         /             \
     2**2             9**2 
     = 4              = 81
        \               /
         \             /
          \           / 
           \         /
             4+81= 85
                |
                |
             Echo 85

# Defining pipeline as a python function

In [3]:
@dsl.pipeline(
  name='P2021PTLP0034-MLOPS',
  description='Implements the above process '
)
def calculate(
    uid = 34
):
  
    step1 = extract_multiply(uid)
    step2 = add_static()
    
    step3 = add(step1.output,step2.output)
    
    step4 = square(step3.output,0)
    step5 = square(step3.output,1)
    
    step6 = add(step4.output,step5.output)
    

    echo_task = echo_op(step6.output)

# Compiling the pipeline

In [4]:
kfp.compiler.Compiler().compile(calculate, 'P2021PTLP0034-MLOPS.zip')

/home/blink/.local/lib/python3.7/site-packages/kfp/components/_data_passing.py:229: UserWarning: Missing type name was inferred as "Integer" based on the value "34".
  .format(type_name, str(value)))
/home/blink/.local/lib/python3.7/site-packages/kfp/dsl/_container_op.py:1264: FutureWarning: Please create reusable components instead of constructing ContainerOp instances directly. Reusable components are shareable, portable and have compatibility and support guarantees. Please see the documentation: https://www.kubeflow.org/docs/pipelines/sdk/component-development/#writing-your-component-definition-file The components can be created manually (or, in case of python, using kfp.components.create_component_from_func or func_to_container_op) and then loaded using kfp.components.load_component_from_file, load_component_from_uri or load_component_from_text: https://kubeflow-pipelines.readthedocs.io/en/stable/source/kfp.components.html#kfp.components.load_component_from_file
  category=FutureWa